In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
from re import sub, search, findall
import os
import pandas as pd
import numpy as np
from glob import glob
def remover(file_):
    file_ = ''.join(file_.split()).lower()
    stop_element = ['менее|<|>|автоматическая|ручная|[()]'
              '|меньш.|больше|более|до(?=[0-9])|менее|iultra|от(?=[0-9])'] #'до' ищет только после чисел, тк идет захват 'биохимисслеДОваниекрови'. аспартатаминОТрансфераза = аспартатаминрансфераза
    for elem in stop_element:
      file_ = sub(elem, '', file_)
    file_ = sub(r',', '.', file_) # иначе не меняет на точку 
    return(file_)
def change_date_format(dt):
    """ на первом этапе переводим "нормальный формат" в формат через дефисы
     на втором этапе ищем вариант дд.мм.гг в формат через дефисы"""
    data_1 =  sub(r'(0?[1-9]|[12]\d|30|31)[.](0?[1-9]|1[0-2])[.](\d{4})', 
                     '\\1-\\2-\\3', dt)
    return sub(r'(0?[1-9]|[12]\d|30|31)[.](0?[1-9]|1[0-2])[.](\d{2})', 
                  '\\1-\\2-20\\3', data_1)

In [3]:
lst = list(glob(os.path.join('/content/drive/MyDrive/parser', '*')))

In [4]:
test_dates = []
patients_id = []
patients_birthday = []
values_fbr = []
for doc in lst:
  with open(doc, encoding='utf-8', errors='ignore') as file_:
#with open('/content/drive/MyDrive/проверка/германович-6.txt') as file_:
    file_ = change_date_format(remover(file_.read()))
    file_1 = sub(r'(гемостазиограмма)(\d{2}-\d{2}-\d{4})', r'\2\1', file_)
    for value in findall(r'(\d{2}-\d{2}-\d{4}гемостазиограмма.*?)(?=\d{2}-\d{2}-\d{4})', file_1):
      date = search(r'(\d{2}-\d{2}-\d{4})(?=гемостазиограмма)', file_1)
      if date:
        test_dates.append(date[0])
      id = search(r'\d+', file_1) #ищем айди. находит самую первую дату в эпикризе. !!! есть эпикризы в которых первые цифры это номер отделения, типа онко отд. №2
      if id:
        patients_id.append(id[0])
      birthday = search("(\d{2})-(\d{2})-(\d{4})", file_1)
      if birthday:
        patients_birthday.append(birthday[0])
      fbr = search(r'(?<=фибриноген)\d+', value)
      if fbr:
        values_fbr.append(fbr[0])
      else:
        values_fbr.append('NaN')  
      

In [5]:
data_dct = {'ID':patients_id, 'Birthday':patients_birthday, 'dates':test_dates, 'fbr':values_fbr}
df = pd.DataFrame(data_dct)
df = df.replace(r'', np.nan, regex=True)
df

,ID,Birthday,dates,fbr
0,4656,14-10-1978,01-08-2018,7
1,4656,14-10-1978,01-08-2018,6
2,4656,14-10-1978,01-08-2018,7
3,1357,04-03-1952,05-03-2020,8
4,7255,16-01-1944,06-11-2017,5
...,...,...,...,...
868,3417,20-04-1936,23-05-2017,3
869,5600,28-11-1964,22-12-2020,6
870,2120,17-01-1969,24-04-2020,10
871,2120,17-01-1969,24-04-2020,9
